In [1]:
import pystac

In [10]:
file_items = pystac.ItemCollection.from_file("rhodes_sentinel-2.json")

In [13]:
for item in file_items:
    print(item.id)
    print(item.datetime)

S2A_35SNA_20230827_0_L2A
2023-08-27 09:00:21.327000+00:00
S2B_35SNA_20230822_0_L2A
2023-08-22 09:00:20.047000+00:00
S2A_35SNA_20230817_0_L2A
2023-08-17 09:00:21.370000+00:00
S2B_35SNA_20230812_0_L2A
2023-08-12 09:00:21.069000+00:00
S2A_35SNA_20230807_0_L2A
2023-08-07 09:00:20.107000+00:00
S2B_35SNA_20230802_0_L2A
2023-08-02 09:00:20.321000+00:00
S2A_35SNA_20230728_0_L2A
2023-07-28 09:00:20.662000+00:00
S2B_35SNA_20230723_0_L2A
2023-07-23 09:00:21.476000+00:00
S2A_35SNA_20230718_0_L2A
2023-07-18 09:00:19.860000+00:00
S2B_35SNA_20230713_0_L2A
2023-07-13 09:00:20.114000+00:00
S2A_35SNA_20230708_0_L2A
2023-07-08 09:00:20.745000+00:00


In [18]:
print(file_items[-1])

<Item id=S2A_35SNA_20230708_0_L2A>


In [19]:
assets = file_items[-1].assets  # last item's asset dictionary
print(assets.keys())

dict_keys(['aot', 'blue', 'coastal', 'granule_metadata', 'green', 'nir', 'nir08', 'nir09', 'red', 'rededge1', 'rededge2', 'rededge3', 'scl', 'swir16', 'swir22', 'thumbnail', 'tileinfo_metadata', 'visual', 'wvp', 'aot-jp2', 'blue-jp2', 'coastal-jp2', 'green-jp2', 'nir-jp2', 'nir08-jp2', 'nir09-jp2', 'red-jp2', 'rededge1-jp2', 'rededge2-jp2', 'rededge3-jp2', 'scl-jp2', 'swir16-jp2', 'swir22-jp2', 'visual-jp2', 'wvp-jp2'])


In [21]:
red_href = assets["red"].href

In [22]:
red_href

'https://sentinel-cogs.s3.us-west-2.amazonaws.com/sentinel-s2-l2a-cogs/35/S/NA/2023/7/S2A_35SNA_20230708_0_L2A/B04.tif'

In [ ]:
items[0].assets.keys()

In [ ]:
print(items[0])

In [ ]:
import rioxarray
raster_rhodes_b9 = rioxarray.open_rasterio(items[0].assets["nir09"].href)


In [ ]:
raster_rhodes_b9

In [ ]:
import rioxarray
red = rioxarray.open_rasterio('red.tif')

In [ ]:
red

In [ ]:
red.values

In [ ]:
red

In [ ]:
red_lowres = rioxarray.open_rasterio('red.tif', overview_level=2)

In [ ]:
red_lowres

In [ ]:
red_lowres.plot()

In [ ]:
red_lowres.plot(robust=True)

## View Raster Coordinate Reference System (CRS) in Python

In [ ]:
red.rio.crs

In [ ]:
epsg = red.rio.crs.to_epsg()
epsg 

In [ ]:
from pyproj import CRS
crs = CRS(epsg)
crs

## Dealing with Missing Data

In [ ]:
red_nodata = rioxarray.open_rasterio('red.tif', masked=True, overview_level=2)

In [ ]:
red_nodata

In [ ]:
red_lowres.where(red_lowres != red_lowres.rio.nodata)

In [ ]:
red_nodata.plot(robust=True)

## Raster Bands

In [ ]:
import pystac
items = pystac.ItemCollection.from_file('rhodes_sentinel-2.json')

In [ ]:
visual_lowres = rioxarray.open_rasterio(items[0].assets['visual'].href, overview_level=2)
visual_lowres

In [ ]:
visual_lowres.shape

In [ ]:
visual_lowres.plot()

In [ ]:
visual_lowres.plot.imshow()